In [94]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

In [161]:
data_list=[]
file_names=['2006-05-13.csv','2006-05-14.csv','2006-05-15.csv','2006-05-16.csv','2006-05-17.csv',
            '2006-05-18.csv','2006-05-19.csv','2007-12-14.csv','2009-05-05.csv','2011-11-01.csv']

for f in file_names:
    data=pd.read_csv('data/raw_data/'+f)
    data = data.drop(['SJX08','SJX13'],axis=1)
    if f in ['2007-12-14.csv','2009-05-05.csv','2011-11-01.csv']:
        q25,q75=np.quantile(data['SJS13'],[0.25,0.75])
        print(q25-1.5*(q75-q25),q75+1.5*(q75-q25))
        data['SJS13'][data['SJS13']<q25-1.5*(q75-q25)]=np.nan#3000
        data['SJS13'][data['SJS13']>q75+1.5*(q75-q25)]=np.nan#2700
    data=data.interpolate(method='linear')
    data=data.iloc[90:-90]
    print(data.shape)
    data_list.append(data)

(172620, 12)
(172620, 12)
(172620, 12)
(172620, 12)
(172620, 12)
(172620, 12)
(172620, 12)
2774.862492801036 2860.8972732559014
(172620, 12)
2785.931150398128 2869.0900732652844
(172620, 12)
2732.015028449452 2869.1944270294844
(172620, 12)


In [178]:
from scipy import signal

In [191]:
def to_lpf(data_list, order, cutoff, fs):
    datas = data_list.copy()
    for i, data in enumerate(datas):
        data = data.values
        data = np.transpose(data)
        datas[i] = data
    cable_data = datas
    
    lpf_list = []
    for i in range(10):
        lpf_list.append([])
        for j in range(12):
            b = signal.firwin(order, cutoff=cutoff, fs=fs, pass_zero='lowpass')
            x = signal.lfilter(b, [1.0], cable_data[i][j])
            lpf_list[i].append(cable_data[i][j][order:] - x[order:])
    return lpf_list

In [203]:
cable_data = to_lpf(data_list, 101, 3, 10)

In [204]:
file_names=['2006-05-13','2006-05-14','2006-05-15','2006-05-16','2006-05-17', '2006-05-18','2006-05-19','2007-12-14','2009-05-05','2011-11-01']
cable_title = ["SJS08", "SJS09", "SJS10", "SJS11", "SJS12", "SJS13", "SJS14", "SJX09", "SJX10", "SJX11", "SJX12", "SJX14"]

In [205]:
for i in range(len(cable_data)):
    df = pd.DataFrame(np.transpose(cable_data[i]), columns=cable_title)
    df.to_csv('data/lpf_data/order=101,cutoff=3,fs=10/' + file_names[i] + '.csv')

In [ ]:
cable_data = to_lpf(data_list, 101, 3, 10)

In [ ]:
#sampling period=60*24*24 sec
#sampling freq = 2 Hz
#Total Sample = 60*24*24*2
#Signal Freq = ?
#Nyquist Freq = 0.5 * 2Hz
#Cutoff = more than signal freq
#order: 차수

$$ N >= log_{10}{\sqrt{{\delta^{-2}_{p}-1} \over {\delta^{-2}_{s}-1}}} / log_{10}{\omega_p\over {\omega_s}} $$